In [1]:
import numpy as np

## extracting revenue 
def true_rev(start,end,root):
    rev = []
    for i in range(start,end):
        rev.append(root[i][0].attrib['yvalue'])
    rev=np.array(rev).astype(np.float)
    return rev

In [2]:
# Function that returns list of bow for all movies in a given set(train/dev/test) of the given file root

from collections import defaultdict

def bow_movie(root,start,end):
    bow_per_movie = [] # list of dict where each element of bow_per_movie is bow for that movie
    review_list=[]
    
    for i in range(start,end):
        bow = defaultdict(float)
        string = ""
        
        for j in range(1,len(root[i])):
            string += root[i][j].text

        tokens =string.split()
        l_tokens = map(lambda t: t.lower(), tokens)

        for token in l_tokens:
            bow[token] += 1.0
        bow_per_movie.append(bow)
        
        review_list.append(string)
    
    return bow_per_movie,review_list


In [3]:
# Function that returns vocab for all movies in the training set of given file root
def vocab(bow_per_movie):
    stop_words=["a","an", "and","are","as","at","be","by","for","from","has","he", "in","is","it","its","of","on","that","the","to","was","were","will","with"]
    
    vocab = defaultdict(float)
    for bow in bow_per_movie:
        for key,value in bow.iteritems():
            vocab[key] += value
            
    ## removing words with frequency less than 5
    for key,value in vocab.items():
        if key in stop_words:
            vocab.pop(key)
    return vocab

In [4]:
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
import scipy
import numpy as np

def gsv(train_x,train_y):
    #parameters = {'max_depth':[1, 1000]}
    #clf = GridSearchCV(DecisionTreeRegressor(), parameters)
    parameter={'alpha': [0.001,100]}
    clf=GridSearchCV(Ridge(),parameters)
    clf.fit(train_x, train_y)
    return clf

def train(train_x,train_y):
    #clf=DecisionTreeRegressor(max_depth=10)
    #clf = ElasticNet(alpha=1.0)
    clf=Ridge(alpha=1)
    clf.fit(train_x, train_y)
    return clf

def predict(clf, test_x): 
    return clf.predict(test_x)

def cal_mae(y_hat,y):
    return np.mean(abs(y_hat-y))


In [5]:
import xml.etree.ElementTree as ET
tree = ET.parse('dataset\\movies-data-v1.0\\movies-data-v1.0\\7domains-train-dev.tl.xml')
root_traindev_to = tree.getroot()

In [6]:
bow_per_movie_train_to, review_list_to = bow_movie(root_traindev_to,0,1147)
#vocab_to=vocab(bow_per_movie_train_to)

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

C:\Users\Deeksha Razdan\Anaconda2\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [8]:
sid = SentimentIntensityAnalyzer()
review_polarity=[0]*len(review_list_to)
i=0
for review in review_list_to:
    ss = sid.polarity_scores(review)
    review_polarity[i]=ss['compound']
    i+=1


In [9]:
from sklearn.neural_network import MLPRegressor
def mpl(train_x,train_y,test_x):
    clf = MLPRegressor(hidden_layer_sizes=(300,300,300,300,300),alpha=0.01)
    clf.fit(train_x, train_y) 
    return clf.predict(test_x)

In [10]:
true_rev_train_to = true_rev(0,1147,root_traindev_to)

In [11]:
review_polarity=np.array(review_polarity).reshape(-1,1)
true_rev_train_to=np.array(true_rev_train_to).reshape(-1,1)

In [15]:
import pickle
f=open('train_y_ps.txt','r')
true_rev_train_ps=pickle.load(f)

In [12]:
trained_classifier=mpl(review_polarity,true_rev_train_to)

TypeError: mpl() takes exactly 3 arguments (2 given)

In [18]:
y_hat=mpl(review_polarity,true_rev_train_to,np.array(review_polarity_dev).reshape(-1,1))

C:\Users\Deeksha Razdan\Anaconda2\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1266: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
cal_mae(y_hat,true_rev_dev_to)

10565521.853055386


#  Predicting revenue in development set

In [13]:
bow_per_movie_dev_to, review_list_dev = bow_movie(root_traindev_to,1147,1464)
true_rev_dev_to = true_rev(1147,1464,root_traindev_to)

In [16]:
f=open('dev_y_ps.txt','r')
true_rev_dev_ps=pickle.load(f)

In [17]:
sid = SentimentIntensityAnalyzer()
review_polarity_dev=[0]*len(review_list_dev)
i=0

## for each review (per movie) calculate the polarity
for review in review_list_dev:
    ss = sid.polarity_scores(review)
    review_polarity_dev[i]=ss['compound']
    i+=1
  

In [35]:
#############################
# insert prediction code here
pred_rev_dev_to = predict(trained_classifier,np.array(review_polarity_dev).reshape(-1,1)) 
mae_rev_dev_to= cal_mae(pred_rev_dev_to,true_rev_dev_to)
############################

print "The mean absolute error for dev set is ", mae_rev_dev_to

The mean absolute error for dev set is  10652492.6587
